In [ ]:
from PIL import Image
import os
import torch
from torchvision import transforms, models
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import snowflake.connector
from dotenv import load_dotenv
load_dotenv()

c:\Users\Ansh Lulla\VS-Code\Agentic_RAG\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [2]:
img_dir = r"C:\Users\Ansh Lulla\VS-Code\Agentic_RAG\Healthcare\healthcare_data\ddidiversedermatologyimages"
img_files = [os.path.join(img_dir, f) for f in os.listdir(img_dir) if f.endswith(".png")]
print(len(img_files))

656


In [3]:
transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

preprocessed_imgs = []
for img_path in img_files:
    img = Image.open(img_path).convert("RGB")
    preprocessed_imgs.append(transform(img))

In [4]:
print(preprocessed_imgs[0])

tensor([[[-0.1999, -0.2342, -0.2342,  ..., -0.8335, -0.7308, -0.7479],
         [-0.1999, -0.2342, -0.2342,  ..., -0.8849, -0.7308, -0.7137],
         [-0.1999, -0.1999, -0.1999,  ..., -0.8164, -0.7993, -0.7308],
         ...,
         [ 0.0227,  0.0398,  0.0227,  ...,  1.2385,  1.2385,  1.2557],
         [-0.0116, -0.0116,  0.0056,  ...,  1.2214,  1.2385,  1.2214],
         [-0.0458, -0.0458, -0.0287,  ...,  1.2214,  1.2385,  1.2214]],

        [[-0.2850, -0.3200, -0.3200,  ..., -1.2304, -1.1429, -1.1604],
         [-0.2850, -0.3200, -0.3025,  ..., -1.2304, -1.1429, -1.1253],
         [-0.2850, -0.2850, -0.2850,  ..., -1.1954, -1.1954, -1.1429],
         ...,
         [-0.4601, -0.4601, -0.4776,  ...,  1.3957,  1.3957,  1.4132],
         [-0.4951, -0.5126, -0.4951,  ...,  1.3782,  1.3957,  1.3782],
         [-0.5301, -0.5301, -0.5301,  ...,  1.3782,  1.3957,  1.3606]],

        [[-0.8981, -0.9678, -1.0027,  ..., -1.5953, -1.5430, -1.5953],
         [-0.9156, -0.9678, -1.0027,  ..., -1

In [5]:
df = pd.read_csv(r"C:\Users\Ansh Lulla\VS-Code\Agentic_RAG\Healthcare\healthcare_data\ddidiversedermatologyimages\ddi_metadata.csv")
df.head()

,Unnamed: 0,DDI_ID,DDI_file,skin_tone,malignant,disease
0,0,1,000001.png,56,True,melanoma-in-situ
1,1,2,000002.png,56,True,melanoma-in-situ
2,2,3,000003.png,56,True,mycosis-fungoides
3,3,4,000004.png,56,True,squamous-cell-carcinoma-in-situ
4,4,5,000005.png,12,True,basal-cell-carcinoma


In [6]:
df.drop("Unnamed: 0", inplace=True, axis=1)

In [7]:
df.head()

,DDI_ID,DDI_file,skin_tone,malignant,disease
0,1,000001.png,56,True,melanoma-in-situ
1,2,000002.png,56,True,melanoma-in-situ
2,3,000003.png,56,True,mycosis-fungoides
3,4,000004.png,56,True,squamous-cell-carcinoma-in-situ
4,5,000005.png,12,True,basal-cell-carcinoma


In [8]:
dff = pd.read_csv(r"C:\Users\Ansh Lulla\VS-Code\Agentic_RAG\Healthcare\healthcare_data\image (2).csv").sort_values(by="ImageID")
dff.head()

,Unnamed: 0,ImageID,Vesicle,Papule,Macule,Plaque,Abscess,Pustule,Bulla,Patch,...,Umbilicated,Poikiloderma,Salmon,Wheal,Acuminate,Burrow,Gray,Pigmented,Cyst,Do not consider this image
603,1203,000001.png,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
638,1268,000002.png,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
277,577,000003.png,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
642,1272,000004.png,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
614,1214,000005.png,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
len(dff), dff["Do not consider this image"].nunique()

(656, 2)

In [10]:
dff.drop("Unnamed: 0", axis=1, inplace=True)

In [11]:
metadata =  dff.set_index("ImageID").to_dict('index')
#metadata["disease"] = df.disease
for k, v in list(metadata.items())[:5]:
    print(k, v)

000001.png {'Vesicle': 0, 'Papule': 0, 'Macule': 1, 'Plaque': 0, 'Abscess': 0, 'Pustule': 0, 'Bulla': 0, 'Patch': 0, 'Nodule': 0, 'Ulcer': 0, 'Crust': 0, 'Erosion': 0, 'Excoriation': 0, 'Atrophy': 0, 'Exudate': 0, 'Purpura/Petechiae': 0, 'Fissure': 0, 'Induration': 0, 'Xerosis': 0, 'Telangiectasia': 0, 'Scale': 0, 'Scar': 0, 'Friable': 0, 'Sclerosis': 0, 'Pedunculated': 0, 'Exophytic/Fungating': 0, 'Warty/Papillomatous': 0, 'Dome-shaped': 0, 'Flat topped': 0, 'Brown(Hyperpigmentation)': 1, 'Translucent': 0, 'White(Hypopigmentation)': 0, 'Purple': 0, 'Yellow': 0, 'Black': 0, 'Erythema': 0, 'Comedo': 0, 'Lichenification': 0, 'Blue': 0, 'Umbilicated': 0, 'Poikiloderma': 0, 'Salmon': 0, 'Wheal': 0, 'Acuminate': 0, 'Burrow': 0, 'Gray': 0, 'Pigmented': 0, 'Cyst': 0, 'Do not consider this image': 0}
000002.png {'Vesicle': 0, 'Papule': 0, 'Macule': 0, 'Plaque': 1, 'Abscess': 0, 'Pustule': 0, 'Bulla': 0, 'Patch': 0, 'Nodule': 0, 'Ulcer': 0, 'Crust': 0, 'Erosion': 0, 'Excoriation': 0, 'Atrophy':

In [12]:
import json
with open("metadata.json", "w") as f:
    json.dump(metadata, f)

In [13]:
# Embedding model
embedding_model = models.resnet50(pretrained=True)
embedding_model

c:\Users\Ansh Lulla\VS-Code\Agentic_RAG\venv\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Ansh Lulla\VS-Code\Agentic_RAG\venv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [14]:
embedding_model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [15]:
feature_extractor = torch.nn.Sequential(*list(embedding_model.children())[:-1])
feature_extractor

Sequential(
  (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace=True)
  (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (4): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)


In [16]:
def generate_embeddings(img_path):
    try:
        img = Image.open(img_path).convert("RGB")
        img = transform(img).unsqueeze(0)
        with torch.no_grad():
            embedding = feature_extractor(img)
            flattened_embeddings = torch.flatten(embedding, start_dim=1)
        
        return flattened_embeddings
    except:
        print(f"Error while creating embedding for {img_path}")
        return None

In [17]:
embedding_sample = generate_embeddings(img_files[0])
embedding_sample.shape

torch.Size([1, 2048])

In [18]:
conn = snowflake.connector.connect(
    user=os.getenv("SNOWFLAKE_USERNAME"),
    password=os.getenv("SNOWFLAKE_PASSWORD"),
    account=os.getenv("SNOWFLAKE_ACC_ID"),
    database="img_data",
    schema="data"
)
cur = conn.cursor()

In [19]:
df.disease.nunique() # number of diseases 

78

In [20]:
pd.Series(df.disease.value_counts() == 1).value_counts() # count of diseases which have only one record in data

count
False    48
True     30
Name: count, dtype: int64

In [21]:
# Generate embeddings and attach metadata
embedding_data = []
for img_path in img_files[:600]:
    img_name = img_path.split("\\")[-1]  
    embedding = generate_embeddings(img_path)
    if embedding is not None and img_name in metadata:
        embedding_data.append({
            "img_name": img_name,
            "embedding": embedding,
            "metadata": metadata[img_name]
        })

In [22]:
embedding_data[0]

{'img_name': '000001.png',
 'embedding': tensor([[0.0961, 0.0566, 0.5163,  ..., 0.2809, 0.9155, 0.1160]]),
 'metadata': {'Vesicle': 0,
  'Papule': 0,
  'Macule': 1,
  'Plaque': 0,
  'Abscess': 0,
  'Pustule': 0,
  'Bulla': 0,
  'Patch': 0,
  'Nodule': 0,
  'Ulcer': 0,
  'Crust': 0,
  'Erosion': 0,
  'Excoriation': 0,
  'Atrophy': 0,
  'Exudate': 0,
  'Purpura/Petechiae': 0,
  'Fissure': 0,
  'Induration': 0,
  'Xerosis': 0,
  'Telangiectasia': 0,
  'Scale': 0,
  'Scar': 0,
  'Friable': 0,
  'Sclerosis': 0,
  'Pedunculated': 0,
  'Exophytic/Fungating': 0,
  'Warty/Papillomatous': 0,
  'Dome-shaped': 0,
  'Flat topped': 0,
  'Brown(Hyperpigmentation)': 1,
  'Translucent': 0,
  'White(Hypopigmentation)': 0,
  'Purple': 0,
  'Yellow': 0,
  'Black': 0,
  'Erythema': 0,
  'Comedo': 0,
  'Lichenification': 0,
  'Blue': 0,
  'Umbilicated': 0,
  'Poikiloderma': 0,
  'Salmon': 0,
  'Wheal': 0,
  'Acuminate': 0,
  'Burrow': 0,
  'Gray': 0,
  'Pigmented': 0,
  'Cyst': 0,
  'Do not consider this im

In [23]:
len(embedding_data)

600

In [24]:
embedding_data[0]["embedding"].squeeze().numpy().shape


(2048,)

In [25]:
# FAISS
embedding_array = np.array([data["embedding"].squeeze() for data in embedding_data]).astype("float32")

d = embedding_array.shape[1]  # Dimension of embeddings
index = faiss.IndexFlatL2(d)
index.train(embedding_array)
index.add(embedding_array)
faiss.write_index(index, "image_embeddings.index")

In [26]:
embedding_array[0].shape

(2048,)

In [27]:
embedding_data[0]["metadata"]

{'Vesicle': 0,
 'Papule': 0,
 'Macule': 1,
 'Plaque': 0,
 'Abscess': 0,
 'Pustule': 0,
 'Bulla': 0,
 'Patch': 0,
 'Nodule': 0,
 'Ulcer': 0,
 'Crust': 0,
 'Erosion': 0,
 'Excoriation': 0,
 'Atrophy': 0,
 'Exudate': 0,
 'Purpura/Petechiae': 0,
 'Fissure': 0,
 'Induration': 0,
 'Xerosis': 0,
 'Telangiectasia': 0,
 'Scale': 0,
 'Scar': 0,
 'Friable': 0,
 'Sclerosis': 0,
 'Pedunculated': 0,
 'Exophytic/Fungating': 0,
 'Warty/Papillomatous': 0,
 'Dome-shaped': 0,
 'Flat topped': 0,
 'Brown(Hyperpigmentation)': 1,
 'Translucent': 0,
 'White(Hypopigmentation)': 0,
 'Purple': 0,
 'Yellow': 0,
 'Black': 0,
 'Erythema': 0,
 'Comedo': 0,
 'Lichenification': 0,
 'Blue': 0,
 'Umbilicated': 0,
 'Poikiloderma': 0,
 'Salmon': 0,
 'Wheal': 0,
 'Acuminate': 0,
 'Burrow': 0,
 'Gray': 0,
 'Pigmented': 0,
 'Cyst': 0,
 'Do not consider this image': 0}

In [28]:
import base64
import os
from mistralai import Mistral

def encode_image(image_path):
    """Encode the image to base64."""
    try:
        with open(image_path, "rb") as image_file:
            return base64.b64encode(image_file.read()).decode('utf-8')
    except FileNotFoundError:
        print(f"Error: The file {image_path} was not found.")
        return None
    except Exception as e: 
        print(f"Error: {e}")
        return None

In [29]:
def search_similar_chunks(query_embedding, top_k=5):
    search_results = conn.search(
        index="image_embeddings",
        embedding=query_embedding,
        top_k=top_k
    )
    return search_results["matches"]

In [30]:
# Test
query_img_path = img_files[642]
query_embedding = generate_embeddings(query_img_path)
image_path = img_files[642]
base64_image = encode_image(image_path)

In [ ]:
# Test 
top_k = search_similar_chunks(query_embedding)

In [31]:
api_key = os.getenv("MISTRAL_API_KEY")
model = "pixtral-12b-2409"  
client = Mistral(api_key=api_key)

In [34]:
messages = [
    {
        "role": "system",
        "content": f"""You are a Medical AI Assistant specializing in analyzing and diagnosing skin diseases. 
The following contextual information has been retrieved from similar cases:
{top_k}

Your goal is to:
1. Accurately identify the skin condition depicted in the provided image.
2. Assess the severity or stage of the condition, if possible.
3. Analyze visible skin features such as color, texture, and irregular patterns.
4. Provide medically relevant observations and recommendations for further diagnosis or treatment.
5. Use the contextual information to provide an even more in-depth report about the disease.

**Medical Report Requirements**:
- Include **Disease Identification**: Clearly state the suspected disease(s) and provide reasoning, citing relevant medical characteristics.
- Detail the **Stage/Severity** of the disease: Include clear explanations about why this stage/severity is suggested based on visible patterns and retrieved information.
- Describe **Skin Characteristics**: Offer detailed descriptions of color, texture, shape, visible patterns, and any noteworthy features.
- Highlight **Patterns of Concern**: Discuss irregularities, such as asymmetry, abnormal texture, or discoloration, and their implications.
- Provide **Treatment Recommendations**: Include a detailed overview of potential treatments, next steps for diagnosis, and preventive measures, where relevant.
- Avoid including retrieved context directly in the report but use it to enhance specificity and accuracy in the analysis.

**Report Format**:
- Use headings and subheadings for each section.
- Provide a detailed explanation of observations in a structured and reader-friendly format.
- Ensure the report reflects a high level of precision, depth, and clinical relevance.
"""
    },
    {
        "role": "user",
        "content": [
            {
                "type": "text",
                "text": """You are provided with a skin disease image for analysis. Please generate a detailed medical report with all the outlined requirements.

                **Image Details**:
                - **Type**: Skin disease image
                - **Format**: Base64-encoded image
                - **Source**: Uploaded by a user for medical assistance

                Please analyze the image and generate a comprehensive report."""
            },
            {
                "type": "image_url",
                "image_url": f"data:image/jpeg;base64,{base64_image}"
            }
        ],
    }
]

# Generate the medical report
chat_response = client.chat.complete(
    model=model,
    messages=messages,
    temperature=0.4,  
    top_p=0.9        
)

print(chat_response.choices[0].message.content)

### Medical Report

#### Patient Information:
- **Image Source**: User-uploaded
- **Image Type**: Skin disease image

---

### Disease Identification:
The image shows a lesion that is suggestive of a **nevus** (mole). The lesion appears to be a benign melanocytic nevus based on its clinical appearance.

#### Reasoning:
- **Appearance**: The lesion is well-circumscribed, round, and has a uniform color.
- **Color**: The lesion is dark brown in color.
- **Size**: The lesion appears to be approximately 0.5 cm in diameter, as indicated by the ruler in the image.

---

### Stage/Severity:
Based on the image, the nevus appears to be a benign and stable lesion. There are no signs of rapid growth, change in color, or irregular borders that would suggest a malignant transformation.

---

### Skin Characteristics:
- **Color**: Dark brown.
- **Texture**: Smooth and uniform.
- **Pattern**: Round and well-circumscribed.
- **Size**: Approximately 0.5 cm in diameter.

---

### Patterns of Concern:
- *